In [18]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain



In [19]:
csv_file_path = 'C:/Users/sachu/Desktop/DB/db_split/db1.csv'
df =pd.read_csv(csv_file_path)

In [20]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')


In [21]:
prompt_template = PromptTemplate(
    template="Compare the following resume and job description, and provide an ATS score out of 100.\n\nResume:\n{Resume_str}\n\nJob Description:\n{description}\n\nATS Score:",
    input_variables=["Resume_str", "description"]

)

In [22]:
def calculate_ats_score(resume, job_description):
    # Encode the resume and job description
    embeddings = model.encode([resume, job_description])
    # Calculate the cosine similarity between the embeddings
    cosine_sim = util.cos_sim(embeddings[0], embeddings[1]).item()
    # Scale the similarity to a score out of 100
    score = int(cosine_sim * 100)
    return score

In [26]:
class CalculateATSScoreChain:
    def __init__(self, model, prompt_template):
        self.model = model
        self.prompt_template = prompt_template

    def run(self, inputs):
        resume = inputs['Resume_str']
        job_description = inputs['description']
        return calculate_ats_score(resume, job_description)

In [27]:
ats_score_chain = CalculateATSScoreChain(model, prompt_template)

In [ ]:
df['ATS_score'] = df.apply(lambda row: ats_score_chain.run({"Resume_str": row['Resume_str'], "description": row['description']}), axis=1)


In [17]:
df.to_csv(csv_file_path, index=False)